In [ ]:
import os
import matplotlib.pyplot as plt

from data import DIV2K
from model.wdsr import wdsr_b
from train import WdsrTrainer

%matplotlib inline

In [ ]:
# Number of residual blocks
depth = 32

# Super-resolution factor
scale = 4

# Downgrade operator
downgrade = 'bicubic'

In [ ]:
# Location of model weights (needed for demo)
weights_dir = "weights/wdsr-b-32-x4"
weights_file = os.path.join(weights_dir, 'weights.h5')

os.makedirs(weights_dir, exist_ok=True)

In [ ]:
model = wdsr_b(scale=scale, num_res_blocks=depth)
model.load_weights(weights_file)

In [ ]:
from tensorflow.keras.losses import MeanAbsoluteError

loss = MeanAbsoluteError()

In [ ]:
from model import resolve_single
from utils import load_image
from tensorflow.keras.metrics import Mean
import numpy as np
import tensorflow as tf
import math

def evaluate(start, end):
    base = ".div2k/images/"
    hrPath = base + "DIV2K_valid_HR/0"
    lrPath = base + "DIV2K_valid_LR_bicubic/X4/0"
    loss_mean = Mean()
    psnr_mean = Mean()
    psnr_mean2 = Mean()
    ssim_mean = Mean()
    for i in range(start, end+1):
        #print(i)
        name = str(800+i)
        lr = load_image(lrPath+name+"x4.png")
        sr = resolve_single(model, lr)
        hr = load_image(hrPath+name+".png")
        psnrValue = tf.image.psnr(hr, sr, max_val=255)
        psnr_mean(psnrValue)
        sr = tf.convert_to_tensor(sr)
        hr = tf.convert_to_tensor(hr)
        sr = tf.cast(sr, tf.float32)
        hr = tf.cast(hr, tf.float32)
        lossValue = loss(hr, sr)
        loss_mean(lossValue)
        psnr2 = 10 * math.log10(255*255/lossValue)
        psnr_mean2(psnr2)
        ssimValue = tf.image.ssim(hr, sr, max_val=255)
        ssim_mean(ssimValue)
        print(i)
    return loss_mean.result(), psnr_mean.result(), psnr_mean2.result(), ssim_mean.result()

lossMean, psnrMean, psnrMean2, ssimMean = evaluate(1, 100)
print("MAE = " + str(lossMean))
print("PSNR = " + str(psnrMean))
print("PSNR2 = " + str(psnrMean2))
print("SSIM = " + str(ssimMean))

In [ ]:
from model import resolve_single
from utils import load_image
import numpy as np
from PIL import Image

def resolve_and_save(lr_image_path, hr_image_path):
    lr = load_image(lr_image_path)
    
    sr = resolve_single(model, lr)
    
    sr = np.array(sr)
    sr = Image.fromarray(sr)
    sr.save(hr_image_path)

In [ ]:
lrPath = ".div2k/images/DIV2K_train_LR_bicubic/X4/0021x4.png"
hrPath = "images/HighRes/0021WDSR.png"
resolve_and_save(lrPath, hrPath)